# Tache ???

## Objectif : créer une table d'analyse centrale qui aggrège toutes les données des différentes tables dont on a besoin.

Les lignes de cette table sont des mesures.

## Lexique

- Prélèvement = un prélèvement d'eau dans le réseau, qui peut servir à tester la présence de plusieurs composants. Identifié par referenceprel.
- Paramètre = un composé chimique, identifiée par cdparametre
- Mesure = la mesure de la concentration d'un paramètre à partir de l'eau d'un prélèvement (qui peut être utilisée pour tester plusieurs paramètres). A 



## Le Rationnel/ la démarche pour la création d'une table d'analyse centrale est la suivante

1) on part de la table des communes et on en extrait leur code insee et la localisation des mesures associées (cdreseau) pour chaque année
2) on utilise la table cog_communes pour ajouter la code département et région à chaque commune
3) on utilise la table des prélevements pour ajouter tous les prélèvements correspondant à chaque commune
4) on utilise la table des résultats pour ajouter toutes les mesures correspondant à chaque prélèvement
5) on ajoute la catégorie du composant pour chaque mesure, à partir de la table mapping_categories
6) on extrait la limite de qualité au format numérique et on ajoute un champ
7) on détermine la conformité de chaque mesure en comparant valtraduite à la limite de qualité

Il y a une commande sql par étape, avec sauvegarde de la table intermédiaire à l'aide de la magic %%sql --with table_etape_precedente --save table_etape

A la fin de ces étapes on obtient une table qui aggrège toutes les données nécessaires pour des analyses ultérieures.


In [25]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Création d'une table d'analyse centrale

## Etape 1 : on part de la table des communes et on en extrait leur code insee et la localisation des mesures associées (cdreseau) pour chaque année

C'est une simple sélection de données dans la table edc_communes

Résultat : table `udi_commune`

In [26]:
%%sql --save udi_commune

SELECT
  edc_communes.inseecommune,
  edc_communes.cdreseau,
  edc_communes.de_partition

FROM
  edc_communes

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,cdreseau,de_partition
01001,001000556,2020
01002,001000369,2020
01004,001000248,2020
01004,001000249,2020
01004,001000251,2020
01005,001000850,2020
01005,001000850,2020
01006,001000235,2020
01007,001000003,2020
01008,001000254,2020


## Etape 2 : on utilise la table cog_communes pour ajouter la code département et région à chaque commune

Question : comment obtenir également les noms des départements et régions ?

Résultat : table `udi_commune_dpt_rg`

In [27]:
%%sql --with udi_commune --save udi_commune_dpt_rg

SELECT
  inseecommune,
  cdreseau,
  udi_commune.de_partition,
  cog_communes.DEP as code_departement,
  cog_communes.REG as code_region
FROM
  udi_commune

LEFT JOIN 
  cog_communes
ON 
  udi_commune.inseecommune == cog_communes.COM


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,cdreseau,de_partition,code_departement,code_region
47171,047000244,2022,47,75
47171,047000246,2022,47,75
47172,047000239,2022,47,75
47173,047000224,2022,47,75
47174,047000240,2022,47,75
47174,047000240,2022,47,75
47175,047000226,2022,47,75
47176,047000007,2022,47,75
47177,047000232,2022,47,75
47178,047000275,2022,47,75


## Etape 3 : on utilise la table des prélevements pour ajouter tous les prélèvements correspondant à chaque commune

Résultat : table `prelevement`

Ici je  réalise une jointure sur cdreseau car on connait le cdreseau de chaque commune et de chaque prélèvement.
Ce c'est probablement pas la meilleure manière de faire.
En effet la jointure à utiliser a été discutée ici : https://outline.services.dataforgood.fr/doc/notes-sur-la-table-edc_prelevements-N7BwMGDZcQ

Et il y a cette PR qui discute ça également : https://github.com/dataforgoodfr/13_pollution_eau/pull/78

A MODIFIER QUAND IL Y AURA UNE REPONSE SATISFAISANTE

In [28]:
%%sql --with udi_commune_dpt_rg --save prelevements

SELECT
  udi_commune_dpt_rg.inseecommune,
  udi_commune_dpt_rg.de_partition AS annee,
  udi_commune_dpt_rg.code_departement,
  udi_commune_dpt_rg.code_region,
  edc_prelevements.referenceprel,
  edc_prelevements.dateprel
FROM 
    udi_commune_dpt_rg
LEFT JOIN
    edc_prelevements 
ON
    udi_commune_dpt_rg.cdreseau = edc_prelevements.cdreseau
    AND udi_commune_dpt_rg.de_partition = edc_prelevements.de_partition


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,code_departement,code_region,referenceprel,dateprel
83119,2020,83,93,08300247884,2020-01-22
83119,2020,83,93,08300247885,2020-01-22
83119,2020,83,93,08300248027,2020-02-04
83119,2020,83,93,08300248030,2020-02-04
83119,2020,83,93,08300248031,2020-02-04
83119,2020,83,93,08300248074,2020-02-04
83119,2020,83,93,08300248352,2020-01-22
83119,2020,83,93,08300248369,2020-02-24
83119,2020,83,93,08300248405,2020-02-04
83119,2020,83,93,08300248484,2020-03-05


## Etape 4 : on utilise la table des résultats pour ajouter toutes les mesures correspondant à chaque prélèvement

Résultat : table `mesures`

Ici j'utilise un INNER JOIN pour ajouter toutes les mesures (différents paramètres) qui correspondent à un prélèvement.

In [29]:
%%sql --with prelevements --save mesures

SELECT 
prelevements.inseecommune,
prelevements.annee,
prelevements.referenceprel,
prelevements.dateprel,
prelevements.code_departement,
prelevements.code_region,
edc_resultats.cdparametre,
edc_resultats.libmajparametre,
edc_resultats.cdparametresiseeaux,
edc_resultats.casparam,
edc_resultats.valtraduite,
edc_resultats.limitequal

FROM
    prelevements
INNER JOIN
    edc_resultats
ON
    prelevements.referenceprel = edc_resultats.referenceprel


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,referenceprel,dateprel,code_departement,code_region,cdparametre,libmajparametre,cdparametresiseeaux,casparam,valtraduite,limitequal
01261,2020,00100119085,2020-02-14,01,84,1161,"DICHLOROÉTHANE-1,2",12DCLE,107-06-2,0.0,<=3 µg/L
01261,2020,00100119085,2020-02-14,01,84,1832,ATRAZINE-2-HYDROXY,A2H,2163-68-0,0.0,"<=0,1 µg/L"
01261,2020,00100119085,2020-02-14,01,84,1457,ACRYLAMIDE,ACRYL,79-06-1,0.0,<=0.1 µg/L
01261,2020,00100119085,2020-02-14,01,84,1036,ACTIVITÉ BÊTA ATTRIBUABLE AU K40,ACTIK40,None,0.034,None
01261,2020,00100119085,2020-02-14,01,84,2098,ACTIVITÉ TRITIUM (3H),ACTITR,None,0.0,None
01261,2020,00100119085,2020-02-14,01,84,1108,ATRAZINE DÉSÉTHYL,ADET,6190-65-4,0.013,"<=0,1 µg/L"
01261,2020,00100119085,2020-02-14,01,84,3159,ATRAZINE DÉSÉTHYL-2-HYDROXY,ADET2,19988-24-0,0.0,"<=0,1 µg/L"
01261,2020,00100119085,2020-02-14,01,84,1830,ATRAZINE DÉSÉTHYL DÉISOPROPYL,ADETD,3397-62-4,0.0,"<=0,1 µg/L"
01261,2020,00100119085,2020-02-14,01,84,1109,ATRAZINE-DÉISOPROPYL,ADSP,1007-28-9,0.0,"<=0,1 µg/L"
01261,2020,00100119085,2020-02-14,01,84,1370,ALUMINIUM TOTAL µG/L,ALTMICR,7429-90-5,0.0,None


## Etape 5 : on ajoute la catégorie du composant pour chaque mesure, à partir de la table mapping_categories

Résultat : table `mesures_cat`

La bonne manière de faire ça est discutée ici https://outline.services.dataforgood.fr/doc/join-entre-edc_resultats-et-mapping_categories-GCSJPLa3od

In [30]:
%%sql --with mesures --save mesures_cat

SELECT
    mesures.inseecommune,
    mesures.annee,
    mesures.referenceprel,
    mesures.dateprel,
    mesures.code_departement,
    mesures.code_region,
    mesures.cdparametre,
    mesures.libmajparametre,
    mesures.cdparametresiseeaux,
    mesures.casparam,
    mesures.valtraduite,
    mesures.limitequal,
    mapping_categories.categorie
FROM
    mesures
LEFT JOIN
    mapping_categories
ON 
  mesures.cdparametre =   mapping_categories.cdparametre
  AND mesures.libmajparametre = mapping_categories.libmajparametre
  AND  mesures.cdparametresiseeaux = mapping_categories.cdparametresiseeaux
  AND  mesures.casparam = mapping_categories.casparam

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,referenceprel,dateprel,code_departement,code_region,cdparametre,libmajparametre,cdparametresiseeaux,casparam,valtraduite,limitequal,categorie
05036,2020,00500110716,2020-10-07,05,93,1700,FENPROPIDIN,FPRO,67306-00-7,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,1765,FLUROXYPIR,FPYR,69377-81-7,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,2547,FLUROXYPIR-MEPTYL,FPYRM,81406-37-3,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,1194,FLUSILAZOL,FSLZ,85509-19-9,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,1908,FURALAXYL,FURALAX,57646-30-7,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,2567,FURATHIOCARBE,FURATHI,65907-30-4,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,1526,GLUFOSINATE,GFST,51276-47-2,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,1506,GLYPHOSATE,GPST,1071-83-6,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,1909,HALOXYFOP-MÉTHYL (R),HALOXYR,72619-32-0,0.0,"<=0,1 µg/L",pesticides
05036,2020,00500110716,2020-10-07,05,93,5508,HALOSULFURON-METHYL,HASULM,100784-20-1,0.0,"<=0,1 µg/L",pesticides


## Etape 6 : on extrait la limite de qualité au format numérique et on ajoute un champ pour l'unite

Résultat : table `mesures_cat_seuil`

ça a été porposé ici https://github.com/dataforgoodfr/13_pollution_eau/pull/70

In [31]:
%%sql --with mesures_cat --save mesures_cat_seuil

SELECT
    inseecommune,
    annee,
    referenceprel,
    dateprel,
    code_departement,
    code_region,
    cdparametre,
    libmajparametre,
    cdparametresiseeaux,
    casparam,
    valtraduite,
    limitequal,
    CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
    regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    categorie,

FROM
    mesures_cat


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,referenceprel,dateprel,code_departement,code_region,cdparametre,libmajparametre,cdparametresiseeaux,casparam,valtraduite,limitequal,limitequal_float,unite,categorie
04058,2020,00400121459,2020-04-09,04,93,1464,CHLORFENVINPHOS,CFVP,470-90-6,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,2950,CHLORFLUAZURON,CHLFLUA,71422-67-8,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,1540,CHLORPYRIPHOS MÉTHYL,CHLPM,5598-13-0,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,1683,CHLOROXURON,CHLX,1982-47-4,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,2938,CINIDON-ETHYL,CINIDON,142891-20-1,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,5481,CINOSULFURON,CINOSUL,94593-91-6,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,7010,CHLORDANE ALPHA,CLAHA,5103-71-9,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,1757,CHLORDANE BÉTA,CLAHB,5103-74-2,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,1132,CHLORDANE,CLAN,57-74-9,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
04058,2020,00400121459,2020-04-09,04,93,1758,CHLORDANE GAMMA,CLAQ,5566-34-7,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides


## Etape 7 : on détermine la conformité de chaque mesure en comparant la concentration du composé à la limite de qualité

A discuter : ici j'ai considéré qu'une valeur de valtraduite = 0 correspondait à un prélèvement conforme, même si ça inclut le cas d'une non quantification.

Résultat : table `mesures_cat_seuil_conformite` (cette table est sauvegardée pour un usage ultérieur dans le notebook)

In [32]:
%%sql --with mesures_cat_seuil --save mesures_cat_seuil_conformite

SELECT
    inseecommune,
    annee,
    referenceprel,
    dateprel,
    code_departement,
    code_region,
    cdparametre,
    libmajparametre,
    cdparametresiseeaux,
    casparam,
    valtraduite,
    limitequal_float,
    unite,
    categorie,
    case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 'non conforme'
        when limitequal_float is not NULL and valtraduite < limitequal_float then 'conforme'
        else 'non analysé'
    end as "resultat"
    
FROM
    mesures_cat_seuil

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,referenceprel,dateprel,code_departement,code_region,cdparametre,libmajparametre,cdparametresiseeaux,casparam,valtraduite,limitequal_float,unite,categorie,resultat
01261,2020,00100119085,2020-02-14,01,84,1832,ATRAZINE-2-HYDROXY,A2H,2163-68-0,0.0,0.10000000149011612,µg/L,métabolite de pesticide,conforme
01261,2020,00100119085,2020-02-14,01,84,1457,ACRYLAMIDE,ACRYL,79-06-1,0.0,0.10000000149011612,µg/L,non classé,conforme
01261,2020,00100119085,2020-02-14,01,84,1108,ATRAZINE DÉSÉTHYL,ADET,6190-65-4,0.013,0.10000000149011612,µg/L,métabolite de pesticide,conforme
01261,2020,00100119085,2020-02-14,01,84,1830,ATRAZINE DÉSÉTHYL DÉISOPROPYL,ADETD,3397-62-4,0.0,0.10000000149011612,µg/L,métabolite de pesticide,conforme
01261,2020,00100119085,2020-02-14,01,84,1109,ATRAZINE-DÉISOPROPYL,ADSP,1007-28-9,0.0,0.10000000149011612,µg/L,métabolite de pesticide,conforme
01261,2020,00100119085,2020-02-14,01,84,1370,ALUMINIUM TOTAL µG/L,ALTMICR,7429-90-5,0.0,None,None,minéral,non analysé
01261,2020,00100119085,2020-02-14,01,84,1369,ARSENIC,AS,7440-38-2,2.0,10.0,µg/L,métaux lourds,conforme
01261,2020,00100119085,2020-02-14,01,84,1107,ATRAZINE,ATRZ,1912-24-9,0.0,0.10000000149011612,µg/L,pesticides,conforme
01261,2020,00100119085,2020-02-14,01,84,1396,BARYUM,BA,7440-39-3,0.036,None,None,non classé,non analysé
01261,2020,00100119085,2020-02-14,01,84,1114,BENZÈNE,BENZ,71-43-2,0.0,1.0,µg/L,hydrocarbure,conforme


## Résultat

La table `mesures_cat_seuil_conformite` est la table d'analyse centrale qui va être utilisée pour la suite